In [3]:
from utils import convert_boardtomatrix
import numpy as np

In [4]:
convert_boardtomatrix(np.array([[1, 0, 1],
                      [0, 0, 0],
                      [0, 0, 0]]))

array([[1., 2., 0.],
       [3., 2., 2.],
       [0., 0., 0.]])